In [7]:
pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
import gurobipy as gp

In [4]:
def get_num_districts(state):

    return None

In [12]:
state = 'RI'
filename = "districting_2020/{}/counties/graph/{}.population".format(state, state)


with open('Numberofdistricts.txt', 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts =  int(parts[1])

with open(filename, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)



In [13]:
# Model
m = gp.Model("district")

state = 'RI'
adjacency_matrix_file = r'districting_2020\2020\RI\counties\graph\RI_distances.csv'

# Set decision variable x for each node j wether it is in district k
for i in range(num_counties):
    for j in range(num_districts):
        x = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

# Set decision variable y for each pair of nodes i and k, and each district j
for i in range(num_counties):
    for k in range(num_counties):
        for j in range(num_districts):
            y = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))







Restricted license - for non-production use only - expires 2025-11-24
